In [1]:
%load_ext autoreload
%autoreload 2

In [70]:
from khmer import Nodegraph
import pysam

import pandas as pd
import seaborn as sns

%matplotlib inline

# Custom scripts
from khtools.compare_peptide import kmerize

In [2]:
Nodegraph.load?

Docstring:
Hashtable.load(type cls, file_name)
Load the graph from the specified file.
Type:      builtin_function_or_method


In [3]:
folder = '/home/olga/data_sm/kmer-hashing/classify_coding_vs_noncoding/'

In [4]:
cd $folder

/home/seqbot/ibm_sm/olga/kmer-hashing/classify_coding_vs_noncoding


In [5]:
ls -lha

total 239G
drwxrwxr-x 2 olga olga 4.0K Sep 14 08:16 ./
drwxrwxr-x 5 olga olga 4.0K Sep 14 08:39 ../
-rw-rw-r-- 1 olga olga  21M Sep 14 06:46 Homo_sapiens.GRCh38.cds.all.fa.gz
-rw-rw-r-- 1 olga olga  16M Sep 14 06:46 Homo_sapiens.GRCh38.ncrna.fa.gz
-rw-rw-r-- 1 olga olga  14M Sep 14 06:46 Homo_sapiens.GRCh38.pep.all.fa.gz
-rw-rw-r-- 1 olga olga  15G Sep 14 07:11 human_cds_k15.nodegraph
-rw-rw-r-- 1 olga olga   65 Sep 14 07:11 human_cds_k15.nodegraph.info
-rw-rw-r-- 1 olga olga  15G Sep 14 07:22 human_cds_k17.nodegraph
-rw-rw-r-- 1 olga olga   65 Sep 14 07:22 human_cds_k17.nodegraph.info
-rw-rw-r-- 1 olga olga  15G Sep 14 07:30 human_cds_k19.nodegraph
-rw-rw-r-- 1 olga olga   65 Sep 14 07:30 human_cds_k19.nodegraph.info
-rw-rw-r-- 1 olga olga  15G Sep 14 07:39 human_cds_k21.nodegraph
-rw-rw-r-- 1 olga olga   65 Sep 14 07:39 human_cds_k21.nodegraph.info
-rw-rw-r-- 1 olga olga  15G Sep 14 07:47 human_cds_k23.nodegraph
-rw-rw-r-- 1 olga olga   65 Sep 14 07:47 human_cds_k23.nodegraph.info
-r

In [109]:
ksize = 21
transcript_type = 'cds'

input_graph = 'human_{transcript_type}_k{ksize}.nodegraph'.format(transcript_type=transcript_type, ksize=ksize)

nodegraph = Nodegraph.load(input_graph)
ksize = nodegraph.ksize()

In [110]:
nodegraph.get?

Docstring:
Hashtable.get(self, kmer)
Retrieve the count for the given k-mer.

        `kmer` can be either a string or an integer representing the hashed
        value of the kmer.

        For Nodetables and Counttables, this function will fail if the
        supplied k-mer contains non-ACGT characters.
        
Type:      builtin_function_or_method


In [111]:
nodegraph.get("A"*ksize)

1

In [112]:
nodegraph.get("T"*ksize)

1

In [115]:
# noncoding_kmer = 'TATGAATTCGTGTTT'

# nodegraph.get(noncoding_kmer)

In [116]:
nodegraph.get("C"*ksize)

0

In [117]:
bam_folder = '/home/olga/ibm_lg/kmer-hashing/brawand2011/nfcore-rnaseq/human/single_end/STAR'
sample_id = 'SRR306838_GSM752691_hsa_br_F_1'
bam_filename = f'{bam_folder}/{sample_id}Aligned.sortedByCoord.out.bam'


In [118]:
%%time

ksize = 15
transcript_types = 'cds', 'ncrna'

transcript_graphs = {}

for transcript_type in transcript_types:

    input_graph = 'human_{transcript_type}_k{ksize}.nodegraph'.format(transcript_type=transcript_type, ksize=ksize)

    transcript_graphs[transcript_type] = Nodegraph.load(input_graph)
transcript_graphs

CPU times: user 0 ns, sys: 2min 11s, total: 2min 11s
Wall time: 5min 5s


In [119]:
lines = []
unique_kmer_lines = []
trancript_type_kmer_lines = []
classification_lines = []

jaccard_minimum = 0.9

verbose = False

with open(bam_filename, 'rb') as f:
    bam = pysam.AlignmentFile(f)
    
    for read in bam:
        jaccards = {}
#         print(read)
        kmers = set(kmerize(read.seq, ksize=ksize))
        n_unique_kmers = len(kmers)
        
        read_number = "R1" if read.is_read1 else "R2"
        read_id = f'{read.qname}_{read_number}'
#         print(f'\nlen(kmers): {n_unique_kmers}')
        unique_kmer_lines.append([read_id, n_unique_kmers])
        if n_unique_kmers < (len(read.seq) - ksize + 1)/2:
            if verbose:
                print(f'Low complexity sequence!!! n_unique_kmers < (len(read.seq) - ksize + 1)/2  --> {n_unique_kmers} < {(len(read.seq) - ksize + 1)/2}')
                print(read_id)
                print(read.seq)
            classification_lines.append([read.qname, 'low complexity'])

            continue
            
    
        for transcript_type, graph in transcript_graphs.items():
            n_kmers_in_type = sum(1 for kmer in kmers if graph.get(kmer) > 0)
#             print(f'\tn kmers in {transcript_type}: {n_kmers_in_type}')
            fraction_in_type = n_kmers_in_type/n_unique_kmers
            trancript_type_kmer_lines.append([read_id, transcript_type, n_kmers_in_type, fraction_in_type])
            jaccards[transcript_type] = fraction_in_type
        if jaccards['cds'] > jaccards['ncrna'] and jaccards['cds'] > jaccard_minimum:
            classification_lines.append([read_id, 'coding'])
        elif jaccards['ncrna'] > jaccards['cds'] and jaccards['ncrna'] > jaccard_minimum:
            classification_lines.append([read_id, 'non-coding'])
        else:
            classification_lines.append([read_id, 'unknown'])
#         print(read.seq)

transcript_type_kmer_df = pd.DataFrame(trancript_type_kmer_lines, columns=['read_id', 'transcript_type', 'n_kmers_in_type', 'fraction_in_type'])
print(transcript_type_kmer_df.shape)
transcript_type_kmer_df.head()

(1825200, 4)


,read_id,transcript_type,n_kmers_in_type,fraction_in_type
0,SRR306838.2504624_R2,cds,8,0.163265
1,SRR306838.2504624_R2,ncrna,6,0.122449
2,SRR306838.13797218_R2,cds,38,1.000000
3,SRR306838.13797218_R2,ncrna,3,0.078947
4,SRR306838.12718178_R2,cds,26,0.684211


In [120]:
read.is_read1

False

In [121]:
len(transcript_type_kmer_df)/2

912600.0

In [122]:
unique_kmer_df = pd.DataFrame(unique_kmer_lines, columns=['read_id', 'unique_kmers'])
print(unique_kmer_df.shape)
unique_kmer_df.head()

(914704, 2)


,read_id,unique_kmers
0,SRR306838.2504624_R2,49
1,SRR306838.13797218_R2,38
2,SRR306838.12718178_R2,38
3,SRR306838.20954835_R2,12
4,SRR306838.15886398_R2,31


In [123]:
classification_df = pd.DataFrame(classification_lines, columns=['read_id', 'classification'])
print(classification_df.shape)
classification_df.head()

(914704, 2)


,read_id,classification
0,SRR306838.2504624_R2,unknown
1,SRR306838.13797218_R2,coding
2,SRR306838.12718178_R2,unknown
3,SRR306838.20954835_R2,coding
4,SRR306838.15886398_R2,coding


In [124]:
classification_df['classification'].value_counts()

unknown           540229
coding            320139
non-coding         52232
low complexity      2104
Name: classification, dtype: int64

In [125]:
transcript_type_kmer_df.head().merge(classification_df.head(), on='read_id')

,read_id,transcript_type,n_kmers_in_type,fraction_in_type,classification
0,SRR306838.2504624_R2,cds,8,0.163265,unknown
1,SRR306838.2504624_R2,ncrna,6,0.122449,unknown
2,SRR306838.13797218_R2,cds,38,1.000000,coding
3,SRR306838.13797218_R2,ncrna,3,0.078947,coding
4,SRR306838.12718178_R2,cds,26,0.684211,unknown


In [126]:
len(transcript_type_kmer_df.read_id.unique())

780184

In [127]:
transcript_type_kmer_df_2d = transcript_type_kmer_df.pivot(index='read_id', columns='transcript_type', values='fraction_in_type')
print(transcript_type_kmer_df_2d.shape)
transcript_type_kmer_df_2d.head()

ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
transcript_type_kmer_df.groupby('read_id')

In [ ]:
transcript_type_kmer_classification = transcript_type_kmer_df.merge(classification_df, on='read_id')#.merge(unique_kmer_df, on='read_id')
print(transcript_type_kmer_classification.shape)
transcript_type_kmer_classification = transcript_type_kmer_classification.drop_duplicates()
print(transcript_type_kmer_classification.shape)
transcript_type_kmer_classification.head()

In [ ]:
transcript_type_kmer_classification.groupby(['classification', 'transcript_type'])['fraction_in_type'].median()

In [ ]:
sns.boxplot(x='fraction_in_type', y='transcript_type', data=transcript_type_kmer_classification, 
            hue='classification', palette='Set2')


In [ ]:
sns.boxplot(x='fraction_in_type', hue='transcript_type', data=transcript_type_kmer_classification, y='classification')


In [ ]:
sns.violinplot(x='fraction_in_type', hue='transcript_type', data=transcript_type_kmer_classification, y='classification')


In [ ]:
g = sns.FacetGrid(data=transcript_type_kmer_classification, hue='classification', col='transcript_type', palette='Set2')
g.map(sns.distplot, 'fraction_in_type')
g.add_legend()

In [ ]:
g = sns.FacetGrid(data=transcript_type_kmer_classification, col='classification', hue='transcript_type', palette='Set2')
g.map(sns.distplot, 'fraction_in_type')
g.add_legend()

## Read in "ground truth" data from alignment

In [ ]:
folder = '/home/olga/data_sm/kmer-hashing/brawand2011/human/'
series = pd.read_csv(f"{folder}/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.read_ids.txt", 
                                   header=None, squeeze=True)
print(series.shape)
series.head()

In [ ]:
classification_df['is_coding'] = classification_df.read_id.isin(series.values)
print(classification_df.shape)
classification_df.head()

In [ ]:
classification_df['true_y'] = classification_df['is_coding'].map(lambda x: 'coding' if x else 'non-coding')

In [ ]:
from sklearn import metrics

In [ ]:
classification_df['classification'].value_counts()

In [ ]:
y_true = classification_df['true_y']
y_pred = classification_df['classification']

labels = ['coding', 'non-coding', 'unknown', 'low complexity']

matrix = metrics.confusion_matrix(y_true, y_pred, labels=labels)
confusion_df = pd.DataFrame(matrix, index=labels, columns=labels)
confusion_df.index.name = 'Ground Truth (Aligned)'
confusion_df.columns.name = 'Prediction (k-mers)'

In [ ]:
confusion_df

In [ ]:
sns.heatmap(confusion_df, cmap='Purples', annot=True)